In [ ]:
import pandas as pd
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, HfFolder
import os

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

# Set the token for use in the huggingface_hub library
HfFolder.save_token(hf_token)

import wandb
from datasets import Dataset

# Replace 'your-api-token' with your actual API token from wandb.ai
wandb.login(key='88bc0f92c0138587605140040c6d0ef652bde63d')

In [ ]:
df = pd.read_csv("/kaggle/input/nlp-mental-health-conversations/train.csv")

In [ ]:
df = df.dropna()

In [ ]:
df

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df

In [ ]:
data = df.to_dict(orient='list')
#data
data = Dataset.from_dict(data)

In [ ]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def preprocess_function(examples):
    # Tokenize the contexts. No need for return_tensors here.
    model_inputs = tokenizer(examples["Context"], max_length=512, truncation=True, padding="max_length")

    # Tokenize the responses. Note: We don't use return_tensors here to avoid the error.
    labels = tokenizer(examples["Response"], max_length=512, truncation=True, padding="max_length")

    # Here's a crucial part: setting -100 for the padding token ids in labels so they're not used in loss calculation
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_example] 
        for label_example in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_data = data.map(preprocess_function, batched=True)

In [ ]:
%pip install evaluate
%pip install rouge_score
%pip install bert-score  # If using BERTScore

In [ ]:
#!pip install -U nltk
#from nltk.translate.meteor_score import meteor_score
import evaluate

'''metrics = evaluate.combine(
    [
        evaluate.load("rouge", use_aggregator=False),
        #evaluate.load("meteor"),  # Adding METEOR
        evaluate.load("bertscore", lang='en'),  # Adding BERTScore
    ]
)'''
metrics = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metrics.compute(predictions=decoded_preds, references=decoded_labels)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    # Adjust rounding to handle lists and non-numeric values
    def safe_round(value, digits):
        if isinstance(value, (int, float)):
            return round(value, digits)
        elif isinstance(value, list):
            return [safe_round(v, digits) for v in value]
        else:
            return value

    rounded_result = {k: safe_round(v, 4) for k, v in result.items()}

    return rounded_result


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
checkpoint = "google-t5/t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
tokenized_data = tokenized_data.train_test_split(test_size=0.25)

In [ ]:
tokenized_data

In [ ]:
print(tokenized_data["train"].features)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="Mental_health_response",
    evaluation_strategy="epoch",  # Change from "epoch" to "steps"
    learning_rate=2e-3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.1,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.push_to_hub()